## Base Model Training

In [1]:
!pip install -q tensorflow-model-optimization

In [1]:
import os
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization
%load_ext tensorboard

In [6]:
# Load Cat vs Dog dataset
(train_ds, val_ds, test_ds), info = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:90%]', 'train[90%:]'], shuffle_files=True, as_supervised=True, with_info=True)

In [7]:
print("Number of  Classes: " + str(info.features['label'].num_classes))
print("Classes : " + str(info.features['label'].names))
NUM_TRAIN_IMAGES = tf.data.experimental.cardinality(train_ds).numpy()
print("Training Images: " + str(NUM_TRAIN_IMAGES))
NUM_VAL_IMAGES = tf.data.experimental.cardinality(val_ds).numpy()
print("Validation Images: " + str(NUM_VAL_IMAGES))
NUM_TEST_IMAGES = tf.data.experimental.cardinality(test_ds).numpy()
print("Testing Images: " + str(NUM_TEST_IMAGES))

Number of  Classes: 2
Classes : ['cat', 'dog']
Training Images: 16283
Validation Images: 4653
Testing Images: 2326


In [8]:
#Defining Batch Size and Input Image Size
batch_size = 32
img_size = [224, 224]

# Resizing Images in Dataset.
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))

#Buffering the Dataset
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
val_ds = val_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)


In [9]:
#Exctracting and Saving Test Images and Labels from Test Dataset
test_images = []
test_labels = []
for image, label in test_ds.take(len(test_ds)).unbatch():
  test_images.append(image)
  test_labels.append(label)

In [8]:
len(test_images)

2326

In [9]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
import numpy as np

model1 = EfficientNetB0(weights='imagenet')

21834768/21834768 [==============================] - 0s 0us/step


In [10]:
model1.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0

In [11]:
model1.compile(
      optimizer=tf.keras.optimizers.Adam(0.0001), loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics = ["accuracy"]
)

In [12]:
# Defining filepath
filepath = 'base_fine_tuned_model.h5'

# Defining Model Save Callback and Reduce Learning Rate Calllback for achieving Better Results.
model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    save_freq="epoch")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=1, min_delta=5*1e-3,min_lr =5*1e-9,)

callback = [model_save, reduce_lr]


In [15]:
# Training the model for 4 epochs total.
model1.fit(train_ds, epochs=4, steps_per_epoch=(len(train_ds)//batch_size), validation_data=val_ds, validation_steps=(len(val_ds)//batch_size), shuffle=False, callbacks=callback)

Epoch 1/4
15/15 [==============================] - ETA: 0s - loss: 6.6237 - accuracy: 0.0104

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 43s 3s/step - loss: 6.6237 - accuracy: 0.0104 - val_loss: 5.9839 - val_accuracy: 0.0078 - lr: 1.0000e-04
Epoch 2/4
15/15 [==============================] - 5s 341ms/step - loss: 4.1669 - accuracy: 0.3313 - val_loss: 2.2039 - val_accuracy: 0.4375 - lr: 1.0000e-04
Epoch 3/4
15/15 [==============================] - 5s 356ms/step - loss: 1.6549 - accuracy: 0.8375 - val_loss: 0.3276 - val_accuracy: 0.9531 - lr: 1.0000e-04
Epoch 4/4
15/15 [==============================] - 6s 389ms/step - loss: 0.6049 - accuracy: 0.9333 - val_loss: 0.2156 - val_accuracy: 0.9766 - lr: 1.0000e-04


In [16]:
#Evaluating the Model on Test Dataset
_, baseline_model_accuracy = model1.evaluate(test_ds, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy*100)

Baseline test accuracy: 97.5494384765625


#Clustering

In [2]:
# Defining Clustered Weigths using TFMOT
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization =tfmot.clustering.keras.CentroidInitialization

#Setting Clustering Parameters
clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

In [3]:
model1 = tf.keras.models.load_model('/content/base_fine_tuned_model.h5')

We will cluster the weights of the entire model, compile the model and print the model summary.

In [4]:
# Cluster a whole model
clustered_model = cluster_weights(model1, **clustering_params)

#Compiling Clustered Model
clustered_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics = ["accuracy"])

#Printing Clustered Model Summary
clustered_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 cluster_rescaling (Cluster  (None, 224, 224, 3)          0         ['input_1[0][0]']             
 Weights)                                                                                         
                                                                                                  
 cluster_normalization (Clu  (None, 224, 224, 3)          7         ['cluster_rescaling[0][0]']   
 sterWeights)                                                                                     
                                                                                     

In [10]:
#Evaluating the Fine-tuned Clustered Model
_, clustered_model_accuracy = clustered_model.evaluate(test_ds, verbose=0)
#print('Baseline Model Test Accuracy:', baseline_model_accuracy*100)
print('Clustered Model Test Accuracy:',clustered_model_accuracy*100)

Clustered Model Test Accuracy: 18.65864098072052


In [ ]:
#Baseline test accuracy: 97.5494384765625

In [ ]:
# Fine-tune model
clustered_model.fit(train_ds,
  batch_size=batch_size,
  epochs=2, validation_data = val_ds)

Epoch 1/2
177/509 [=========>....................] - ETA: 1:42 - loss: 9.3785 - accuracy: 0.6684